## *Import Necessary Libraries*

In [52]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

from bs4 import  BeautifulSoup
import requests, pandas as pd

In [2]:
#site url
website = "https://imdb.com"

# *Selenium Working*

In [4]:
#initialize web driver
driver = webdriver.Chrome("C:\webDrivers\chromedriver.exe")
#load the website in driver
driver.get(website)
#maximize the driver window
driver.maximize_window()
#sleeps for 1 second
time.sleep(3)

In [5]:
#find the dropdown to click
dropdown = driver.find_element(By.CLASS_NAME, 'ipc-icon--arrow-drop-down')
#click dropdown
dropdown.click()
time.sleep(1)

#find the advanced search link
# adv_search = driver.find_element(By.XPATH, '//a[@class="ipc-list__item searchCatSelector__item"]')
adv_search = driver.find_element(By.XPATH, "//*[contains(text(),'Advanced Search')]")
#click the advance search link
adv_search.click()

#advance search
adv_title_search = driver.find_element(By.XPATH, "//*[contains(text(),'Advanced Title Search')]")
adv_title_search.click()

#click feature film
feature_film = driver.find_element(By.XPATH, "//*[contains(text(),' Feature Film')]")
feature_film.click()

#click tv movie
tv_movie = driver.find_element(By.XPATH, "//*[contains(text(),' TV Movie')]")
tv_movie.click()


#click and type minimum date
minimum_date = driver.find_element(By.XPATH, "//*[@name='release_date-min']")
minimum_date.click()
minimum_date.send_keys('2000')


#click and type maximum date
maximum_date = driver.find_element(By.XPATH, "//*[@name='release_date-max']")
maximum_date.click()
maximum_date.send_keys('2012')


#click and type minimum rating
minimum_rating = driver.find_element(By.XPATH, "//*[@name='user_rating-min']")
minimum_rating.click()

dropdown_2 = Select(minimum_rating)
dropdown_2.select_by_visible_text('3.1')


#click and type maximum rating
maximum_rating = driver.find_element(By.XPATH, "//*[@name='user_rating-max']")
maximum_rating.click()

dropdown_3 = Select(maximum_rating)
dropdown_3.select_by_visible_text('7.8')

#click oscar nominatead
oscar_nominated = driver.find_element(By.XPATH, "//*[contains(text(),'Oscar-Nominated')]")
oscar_nominated.click()

#pick color
color = driver.find_element(By.XPATH, "//input[@id='colors-1']")
color.click()

#pick languages
language = driver.find_element(By.XPATH, "//select[@name='languages']")

select_language = Select(language)
select_language.select_by_visible_text("English")

#select 250 results
results_count = driver.find_element(By.XPATH, "//*[@id='search-count']")

select_results_count = Select(results_count)
select_results_count.select_by_visible_text("250 per page")

#click search button
search_button = driver.find_element(By.XPATH, "//p/button")
search_button.click()

#current_url
current_url = driver.current_url

#quit the driver
driver.quit()

# *Request & Beautiful Soup Working*

In [58]:
#use request library to do a get request on the current url
req = requests.get(current_url)

In [59]:
#200 means request was success
req.status_code

200

In [60]:
#soup object
soup = BeautifulSoup(req.content, 'html.parser')

In [61]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film/TV Movie,
Released between 2000-01-01 and 2012-12-31,
User Rating between 3.1 and 7.8,
Oscar-Nominated,
Color,
English
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/tit

In [62]:
#results
list_items = soup.find_all('div', {'class' : 'lister-item'})

In [63]:
len(list_items)

250

### Data We Need To Extract|

- Movie Title
- Movie Year
- Movie Duration
- Movie Genre
- Movie Rating

In [64]:
#movie name
list_items[0].find('h3').find('a').get_text()

#movie year
list_items[0].find('h3').find('span', {"class" : 'lister-item-year'}).get_text().replace('(', '').replace(')', '')

#movie duration
list_items[0].find('p').find('span', {"class": "runtime"}).get_text()

#movie genre
list_items[0].find('p').find('span', {"class": "genre"}).get_text().replace('\n', '').replace('  ', '')

#movie genre
list_items[0].find('div', {"class": "lister-item-content"}).find('div', {"class": "ratings-bar"}).find('div').find('strong').get_text()

'7.6'

In [65]:
#Title List
title_list = [ result.find('h3').find('a').get_text() for result in list_items ]

#year List
year_list = [ result.find('h3').find('span', {"class" : 'lister-item-year'}).get_text().replace('(', '').replace(')', '') for result in list_items ]

#duration List
duration_list = [ result.find('p').find('span', {"class": "runtime"}).get_text() for result in list_items ]

#genre List
genre_list = [ result.find('p').find('span', {"class": "genre"}).get_text().replace('\n', '').replace('  ', '') for result in list_items ]

#rating List
rating_list = [ result.find('div', {"class": "lister-item-content"}).find('div', {"class": "ratings-bar"}).find('div').find('strong').get_text() for result in list_items ]



### Create Dataframe and CSV

In [66]:
df = pd.DataFrame({
    "Title" : title_list,
    "Year" : year_list,
    "Duration" : duration_list,
    "Genre" : genre_list,
    "Rating" : rating_list,
})

In [67]:
df.head()

,Title,Year,Duration,Genre,Rating
0,Harry Potter and the Sorcerer's Stone,2001,152 min,"Adventure, Family, Fantasy",7.6
1,Spider-Man,2002,121 min,"Action, Adventure, Sci-Fi",7.4
2,Flight,I 2012,138 min,"Drama, Thriller",7.3
3,Troy,2004,163 min,"Drama, History",7.3
4,Avatar,2009,162 min,"Action, Adventure, Fantasy",7.8


In [76]:
df['Year'] = df['Year'].apply(lambda x : x.strip('I'))

In [77]:
df.head()

,Title,Year,Duration,Genre,Rating
0,Harry Potter and the Sorcerer's Stone,2001,152 min,"Adventure, Family, Fantasy",7.6
1,Spider-Man,2002,121 min,"Action, Adventure, Sci-Fi",7.4
2,Flight,2012,138 min,"Drama, Thriller",7.3
3,Troy,2004,163 min,"Drama, History",7.3
4,Avatar,2009,162 min,"Action, Adventure, Fantasy",7.8


In [78]:
df.to_csv('idmb_dataset.csv')